In [33]:
#Set Up Dependancies
import pandas as pd
import requests
import time
from config import nyt_api_key
from scipy.stats import linregress

# Grabbing Numbers of Environmental Articles

In [2]:
#Set url for api call
base_url = "https://api.nytimes.com/svc/search/v2/articlesearch.json"

#define initial parameters
params = {
    'api-key': nyt_api_key,
    'q':'Environment OR climate'
}

#define empty array for article counts
nyt_arts_list = []

for y in range(2001, 2022):
    
    #add to our dictionary of params
    params['begin_date'] = f"{y}0101"
    params['end_date'] = f"{y}1231"
    
    #get response from first query
    response = requests.get(base_url, params=params).json()
    
    #add to climate change list of articles
    nyt_arts_list.append(response['response']['meta']['hits'])
    
    #add a sleep between iterations to avoid getting banned, whoops
    time.sleep(15)

#printing out list of articles for reference
nyt_arts_list

[297,
 322,
 261,
 232,
 329,
 684,
 1121,
 1121,
 1256,
 724,
 636,
 509,
 472,
 530,
 641,
 588,
 1049,
 1254,
 1510,
 1379,
 1590]

In [60]:
#Create Dataframe 
nyt_df = pd.DataFrame({
    "Years":range(2001,2022),
    "#Articles":nyt_arts_list
})

In [7]:
#output to CSV for visualization
nyt_df.to_csv("Resources/nyt_arts_list_by_year.csv")

# Getting Energy Production Data

In [8]:
#contributed by Jennifer Vincent
energy_data = "Resources/EnergyJV.csv"
energy_data_df = pd.read_csv(energy_data)
energy_data_df = energy_data_df.drop(columns = ['units','source key'])
# energy_data_df

In [9]:
#As the data is organized horizonally, we'll transpose the data so that the rows become columns.

#transpose function
energy_data_t = energy_data_df.transpose()
#move top row as column heads
energy_data_t.set_axis(energy_data_t.iloc[0], axis=1, inplace=True)
#Remove top row
energy_data_list = energy_data_t[1:]
#remove unnecessary columns
energy_data_list = energy_data_list['United States : all fuels (utility-scale)']


In [10]:
#Grabbing energy by month for visualizing later
energy_by_month = energy_data_list

In [11]:
#Calulating Total Emission by summing 12 months of the year. 
us_eo_by = []
temp_sum = 0
m_count = 0
y_count = 0
for m in range(len(energy_data_list)):
    temp_sum = temp_sum + int(energy_data_list[m])
    m_count = m_count + 1  
    if m_count == 12:
         
        us_eo_by.append(temp_sum)
        m_count = 0
        temp_sum = 0
        y_count = y_count + 1
        
        if y_count == 22:
            break

In [36]:
#Create DF for easier graphing
us_eo_by_df = pd.DataFrame({
    "Years": range(2001,2022),
    "Energy Production (thousand megawatthours)": us_eo_by
})

In [37]:
#Sending to CSV for visualization
us_eo_by_df.to_csv("Resources/us_total_energy_out.csv")

# CO2 production Data

In [16]:
#Contributed by Jennifer Vincent
#pulling in CO2 data
co2_data = "Resources/CO2_emission_by_countries.csv"
co2_data_df = pd.read_csv(co2_data, encoding="ISO-8859-1")

In [22]:
#Using Loc to specify US data, and years after 2001
us_co2_df = co2_data_df.loc[(co2_data_df['Code'] == 'US') & (co2_data_df['Year'] >= 2001 )]
#removing unnecessary columns
us_co2_df = us_co2_df[['Year','CO2 emission (Tons)']]

In [25]:
#Sending to CSV for visualization
us_co2_df.to_csv("Resources/us_co2_vs_year.csv", index=False)

# Linear Regression Data

In [59]:
#Read in CSVs to dataframes
path = "Resources/us_total_energy_out.csv"
us_energy_df = pd.read_csv(path, index_col=0)

path = "Resources/us_co2_vs_year.csv"
us_co2_df = pd.read_csv(path)

path = "Resources/meant_vs_year.csv"
us_t_df = pd.read_csv(path)


In [46]:
#setting up lin regress values and equation for Energy Output
x = us_energy_df['Years']
y = us_energy_df['Energy Production (thousand megawatthours)']
(slope,intercept,rvalue,pvalue,stderr) = linregress(x,y)

In [54]:
#Predictions for Energy Output by the US
pred_2033 = (slope * 2033 + intercept)/1000000
pred_2053 = (slope * 2053 + intercept)/1000000
pred_2123 = (slope * 2123 + intercept)/1000000
print(f"In 2033, the United States is on track to create {round(pred_2033 ,2)} billion megawatthours of energy.")
print(f"In 2053, the United States is on track to create {round(pred_2053, 2)} billion megawatthours of energy.")
print(f"In 2123, the United States is on track to create {round(pred_2123, 2)} billion megawatthours of energy.")

In 2033, the United States is on track to create 492078.2 billion megawatthours of energy.
In 2053, the United States is on track to create 604769.92 billion megawatthours of energy.
In 2123, the United States is on track to create 999190.98 billion megawatthours of energy.


In [49]:
#setting up lin regress values and equation for CO2 Output
x = us_co2_df['Year']
y = us_co2_df['CO2 emission (Tons)']
(slope,intercept,rvalue,pvalue,stderr) = linregress(x,y)

In [53]:
#Predictions for CO2 output by the US
pred_2033 = (slope * 2033 + intercept)/1000000000
pred_2053 = (slope * 2053 + intercept)/1000000000
pred_2123 = (slope * 2123 + intercept)/1000000000
print(f"In 2033, the United States is on track to emit {round(pred_2033 ,1)} billion tons of C02.")
print(f"In 2053, the United States is on track to emit {round(pred_2053, 1)} billion tons of C02.")
print(f"In 2123, the United States is on track to emit {round(pred_2123, 1)} billion tons of C02.")

In 2033, the United States is on track to emit 492.1 billion tons of C02.
In 2053, the United States is on track to emit 604.8 billion tons of C02.
In 2123, the United States is on track to emit 999.2 billion tons of C02.


In [57]:
#setting up lin regress values and equation for CO2 Output
x = us_t_df['Year']
y = us_t_df['Temp (F)']
(slope,intercept,rvalue,pvalue,stderr) = linregress(x,y)

In [58]:
#Predictions for Average US temperature
pred_2033 = slope * 2033 + intercept
pred_2053 = slope * 2053 + intercept
pred_2123 = slope * 2123 + intercept
print(f"In 2033, the United States is on track to have an average temperate of {round(pred_2033 ,1)} F.")
print(f"In 2053, the United States is on track to have an average temperate of {round(pred_2053, 1)} F.")
print(f"In 2123, the United States is on track to have an average temperate of {round(pred_2123, 1)} F.")

In 2033, the United States is on track to have an average temperate of 58.0 F.
In 2053, the United States is on track to have an average temperate of 61.4 F.
In 2123, the United States is on track to have an average temperate of 73.3 F.
